In [ ]:
# SaaS Product Analytics: User Retention Dashboard
This notebook analyzes user activity and retention for a mock SaaS app.


In [1]:
import pandas as pd

# Load dataset
df = pd.read_csv('saas_user_retention_dataset.csv', parse_dates=['signup_date', 'last_active_date'])
df.head()


,user_id,signup_date,last_active_date,feature_used,session_count,plan_type,country,churned
0,U1000,2025-04-07,2025-04-07,Feature A,4,Free,DE,No
1,U1001,2025-04-04,2025-04-22,Feature B,3,Basic,US,No
2,U1002,2025-04-13,2025-04-25,Feature A,5,Pro,IN,Yes
3,U1003,2025-04-15,2025-04-27,Feature C,8,Free,US,No
4,U1004,2025-04-11,2025-04-14,Feature C,4,Pro,DE,No


In [ ]:
## Daily Active Users (DAU)


In [2]:
dau_df = df.groupby('last_active_date').agg(daily_active_users=('user_id', 'nunique')).reset_index()
dau_df.head()


,last_active_date,daily_active_users
0,2025-04-01,4
1,2025-04-02,10
2,2025-04-03,7
3,2025-04-04,6
4,2025-04-05,8


In [ ]:
## Churn Rate by Plan


In [3]:
churn_by_plan = df.groupby(['plan_type', 'churned']).size().unstack().fillna(0)
churn_by_plan['churn_rate_%'] = churn_by_plan['Yes'] / (churn_by_plan['Yes'] + churn_by_plan['No']) * 100
churn_by_plan[['churn_rate_%']]


churned,churn_rate_%
plan_type,
Basic,28.092784
Free,27.604167
Pro,31.140351


In [ ]:
## Feature Usage Funnel


In [4]:
feature_funnel = df['feature_used'].value_counts().reset_index()
feature_funnel.columns = ['feature', 'user_count']
feature_funnel


,feature,user_count
0,Feature A,342
1,Feature C,337
2,Feature B,321


In [ ]:
## Cohort Analysis - Retention


In [5]:
df['signup_week'] = df['signup_date'].dt.to_period('W').apply(lambda r: r.start_time)
df['days_active'] = (df['last_active_date'] - df['signup_date']).dt.days
cohort_retention = df.groupby(['signup_week', 'days_active']).size().unstack(fill_value=0)
cohort_retention.head()


days_active,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
signup_week,,,,,,,,,,,,,,,,,,,,,
2025-03-31,17,15,13,14,13,14,11,17,9,11,...,12,16,16,13,14,22,12,4,13,21
2025-04-07,14,20,14,10,22,15,18,22,13,18,...,12,20,17,17,12,9,16,16,14,16
2025-04-14,5,5,4,7,3,5,0,3,2,2,...,6,1,8,2,10,7,4,4,4,4


In [6]:
# Save outputs for Tableau
dau_df.to_csv('dau_data.csv', index=False)
churn_by_plan.to_csv('churn_data.csv')
feature_funnel.to_csv('funnel_data.csv', index=False)
cohort_retention.to_csv('cohort_retention.csv')
